In [12]:
import sys
sys.path.append("../../")

import torch
import torchvision.utils
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary

import numpy as np
import cv2
import pandas as pd
from sklearn.metrics import accuracy_score

import src.neuralNetworksArch.BstCnn as bst
import src.neuralNetworksArch.Mpkp as mpkp

import src.dataPreparation.CreatePartial as create_partial
import src.utils.Visual as vis
import src.utils.Checkpoint as ckp
import src.utils.Metrics as metrics

from src.config.Param import *

In [49]:
MODEL_PATH = '../../models/FULL - BST - CUHK02.pth'
PATH = '../../dataset/testing/diff_cam/'
DATATEST_PATH = PATH + 'testing.csv'
IMAGES_PATH = PATH + '/images/full/'
IMAGES_OCCL_PATH = PATH + '/images/occl_60/'

In [50]:
def cv_image2tensor(img, convert=True):
    if convert:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img[:, :, ::-1].transpose((2, 0, 1)).copy()
    img = torch.from_numpy(img).float() / 255.0
    img = [img]
    img_tensors = torch.stack(img)

    return img_tensors

In [51]:
model = {}
model_arch  = bst.BstCnn()
checkpoint = ckp.load_checkpoint(load_dir=MODEL_PATH)
model_arch.load_state_dict(checkpoint['state_dict'])
model_arch.eval()

model['model'] = model_arch
model['min_dist'] = checkpoint['dist'][0]
model['max_dist'] = checkpoint['dist'][1]
# model['threshold'] = checkpoint['threshold'] + 0.1
model['threshold'] = 0.5
model['threshold_list'] = checkpoint['threshold_list']

In [ ]:
df = pd.read_csv(DATATEST_PATH)
y_true = []
y_pred = []

verbose = False
with torch.no_grad():
    for index, data in df.iterrows():
        if verbose == True:
            print('No-{}'.format(index+1))
            print('-'*50)
        img1 = cv2.imread(IMAGES_PATH + data['image_1'])
        img2 = cv2.imread(IMAGES_OCCL_PATH + data['image_2'])
        label = data['label']
        
        input1 = cv_image2tensor(img1)
        input2 = cv_image2tensor(img2)
        out1, out2 = model['model'](input1, input2)
        euclidean_distance = F.pairwise_distance(out1, out2)

#         dist = metrics.normalize_data(euclidean_distance.item(), model['max_dist'])
        dist = 1 - (1 / (1 + euclidean_distance.item()))
        pred = 0.0 if dist <= model['threshold'] else 1.0
        y_pred.append(pred)
        y_true.append(float(label))
        
        if verbose == True:
            print('actual distance => {}'.format(dist))
            print('actual thresh => {}'.format(model['threshold']))
            print('actual label => {}'.format(label))
        
        if verbose == True:
            concatenated = torch.cat((cv_image2tensor(img1, False), cv_image2tensor(img2, False)),0)
            vis.imshow(torchvision.utils.make_grid(concatenated))
            print('='*50, end='\n\n')

acc = accuracy_score(np.array(y_true), np.array(y_pred))
print('Accuracy : {}'.format(acc))